# V2：加入bend_penalty，且每个格子不是单元cost

In [21]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from queue import Queue
filepath_out='../output/bench2.router'
netlist_file_path = '../benchmark/bench2.nl'
gridfile_path  = '../benchmark/bench2.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_gird_original,layer2_grid = parse_gridfile(gridfile_path)

bend_penalty: 50 
 via_penalty: 0
size of layer1: (40, 200)
size of layer2: (40, 200)


In [22]:
def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid, path):
    for cell in path:
        x, y, _ = cell
        layer1_grid[x][y] = -1

In [23]:
def get_cell_cost(layer1_grid, cell,path_tmp,bend_penalty):
    x, y, layer = cell
    cell_cost = 1  # 默认的单元代价
    if layer1_grid[x][y] == -1:
        cell_cost = float('inf')  # -1表示无法通过的细胞
    elif layer1_grid[x][y] != 0:
        cell_cost = layer1_grid[x][y]  # 非单元代价
    if len(path_tmp) >= 2:
        prev_cell = path_tmp[-2]
        prev_x, prev_y, prev_layer = prev_cell
        if layer != prev_layer and (prev_x != x or prev_y != y):  # 方向变化
            cell_cost += bend_penalty
    return cell_cost

def expand_source_to_target(rows, columns, layer1_grid, source, target,bend_penalty):
    queue = Queue()
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    queue.put(source_tuple)
    visited.add(source_tuple)
    costs[source_tuple] = 0  # Initial cost for the source cell is 0

    while not queue.empty():
        current_cell = queue.get()

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,cost

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])
            path_tmp= reconstruct_path(source_tuple, current_cell, parents)
            # Calculate the cost to reach the neighbor cell
            cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty)

            if neighbor_tuple not in visited or cost < costs[neighbor_tuple]:
                queue.put(neighbor_tuple)
                visited.add(neighbor_tuple)
                parents[neighbor_tuple] = current_cell
                costs[neighbor_tuple] = cost
    return []

def get_neighbors(rows, columns, cell):
    x, y, layer = cell
    neighbors = []
    if x > 0:
        neighbors.append({'x': x - 1, 'y': y, 'layer': layer})
    if x < columns - 1:
        neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
    if y > 0:
        neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
    if y < rows - 1:
        neighbors.append({'x': x, 'y': y + 1, 'layer': layer})

    return neighbors

In [12]:
def nounit_bend_penalty_cost_router(rows, columns, layer1_grid, nets,bend_penalty):
    routing_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = -1
        layer1_grid[pin2['x']][pin2['y']] = -1
        ## 防止布线在后续的pin上，先将所有的pin标记为-1；
        
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = 1
        layer1_grid[pin2['x']][pin2['y']] = 1
        path = expand_source_to_target(rows, columns, layer1_grid, pin1, pin2,bend_penalty)
        mark_path_on_grid(layer1_grid,path)
        routing_table[net_id] = path
    return routing_table



In [24]:
layer1_gird = layer1_gird_original
routing_table = {}
net_id = nets[0]['net_id']
pin1 = nets[0]['pin1']
pin2 = nets[0]['pin2']
layer1_gird[pin1['x']][pin1['y']] = 1
layer1_gird[pin2['x']][pin2['y']] = 1
path,cost = expand_source_to_target(rows, columns, layer1_gird, pin1, pin2,bend_penalty)
mark_path_on_grid(layer1_gird,path)
routing_table[net_id] = path